In [116]:
import pandas as pd
import numpy as np
import seaborn as sns

import plotly as py
import plotly.io as pio
import plotly.graph_objs as go
from plotly.subplots import make_subplots

from plotly.offline import init_notebook_mode
# Show charts when running kernel
init_notebook_mode(connected=True)

df = pd.read_csv('spy.us.txt')
df['Close_diff']=df['Close'].diff()
df.head()

,Date,Open,High,Low,Close,Volume,OpenInt,Close_diff
0,2005-02-25,104.77,106.00,104.68,105.79,70221808,0,NaN
1,2005-02-28,105.55,105.68,104.56,105.08,79695344,0,-0.71
2,2005-03-01,105.22,105.87,105.22,105.62,54607412,0,0.54
3,2005-03-02,105.21,106.22,105.10,105.57,73733090,0,-0.05
4,2005-03-03,105.99,106.20,105.15,105.61,71286823,0,0.04


# 1. RSI
# $ \frac{Avg(PriceUp)}{Avg(PriceUP)+Avg(PriceDown)}*100 $
###  PriceUp(t) = $ Price(t)-Price(t-1)$ `price > 0` 

###  PriceDown(t) = $ abs((Price(t)-Price(t-1)))$`price < 0`


In [117]:
def RSI(df, n=14):
    delta = df['Close_diff']
    pricesUp   = delta.copy()
    pricesDown = delta.copy()
    
    pricesUp[pricesUp < 0] = 0
    pricesDown[pricesDown > 0] = 0
    
    rollUp   = pricesUp.rolling(n).mean()
    rollDown = -1*pricesDown.rolling(n).mean()
    rsi = 100 * rollUp / (rollUp + rollDown)
    return rsi

In [118]:
df['RSI_14D'] = RSI(df).fillna(0)

# 2. Bollinger Bands
1. Средняя линия ***Avg*** *(обычное скользящее среднее)* 
2. Верхняя линия ***TL*** (средняя линия ***Avg***, смещенная вверх на определенное число ***D*** стандартных отклонений Std)
3. Нижняя линия ***BL*** (средняя линия ***Avg***, смещенная вниз на число стандартных ***D*** отклонений Std)
### $TL|BL = Avg_{30} ± (D * Std_{30})$

[Std](https://youtu.be/GqMOty05n44) – стандартное отклонение рассчитывается как:

   ### $Std = \sqrt{\frac{\sum(Price_i — Avg_{30})^{2}}  {N}}$


In [119]:
def bbands(df, length=30, numsd=2):
    """ returns MA, TL, and BL"""
    close = df['Close']
    ave = close.rolling(length).mean()
    std  = close.rolling(length).std()
    upband = ave + (std * numsd)
    dnband = ave - (std * numsd)
    return np.round(ave,3), np.round(upband,3), np.round(dnband,3)

In [120]:
df = df.assign(**dict(zip(
                ('BB_Middle_Band', 'BB_Upper_Band', 'BB_Lower_Band'),
                bbands(df))))

# 3. Aroon Oscillator [AO](https://equity.today/indikator-aroon.html)  
### $ AO = AroonUp − AroonDown $
### $ AroonUp|Down = 100*\frac{25-PeriodsSince(High|Low)_{25}}{25} ,\;25 =\;количеству\: периодов$

In [121]:
# idx_local_high= количеству дней от начала до локального макс/мин
# если макс/мин сегодня, то переменная равна N
# если макс/мин первый день, то переменная равна 1 

def aroon(df, N=25):
    aroonup = []
    aroondown = []
    x = N
    while x<= len(df['Date']):
        slice_h =list(df['High'][x-N:x])
        slice_l =list(df['Low'][x-N:x])
        
        idx_local_high = 1 + slice_h.index(max(slice_h))
        idx_local_low =  1 + slice_l.index(min(slice_l))
        
        aroonup.append(idx_local_high/N*100)
        aroondown.append(idx_local_low/N*100)
        x+=1
    return aroonup, aroondown

In [122]:
listofzeros = [None] * 24
up, down = aroon(df)
aroon_list = [x - y for x, y in zip(up,down)]
df['Aroon_25D'] = listofzeros+aroon_list

# 4. Price Volume Trend [PVT](https://www.metastock.com/customer/resources/taaz/default.aspx?p=93)
### $PVT = [\frac{CurrentClose - PreviousClose} {PreviousClose} * Volume] + PreviousPVT $

In [1]:
def pvt(df):
    df['PVT']=None
    PreviousPVT=0
    for stock in range(1,len(df)):
        PVT =   df.loc[stock, 'Close_diff'] / df.loc[stock-1, 'Close'] * df.loc[stock, 'Volume']
        df.loc[stock, 'PVT'] = PVT + PreviousPVT
        PreviousPVT = PVT

In [124]:
pvt(df)

# 5. Acceleration Bands [AB](https://fx-roboti.ru/acceleration-bands.html)
### $Upper Band =SMA(High* ( 1 ±^{?} 4 *\frac{High - Low}{High + Low})) $
### $Middle Band = SMA_{}$
### $Lower Band = SMA(Low *
 (1 - 4 * \frac{High - Low}{High + Low}))$
- SMA =  Simple Moving Average

In [125]:
def abands(df):
    df['AB_Middle_Band'] = df['Close'].rolling(20).mean()
    df['ab_up']  = df['High'] * (1 + 4 * (df['High']-df['Low'])/(df['High']+df['Low']))
    df['ab_down'] = df['Low'] * (1 - 4 * (df['High']-df['Low'])/(df['High']+df['Low']))

    df['AB_Upper_Band'] = df['ab_up'].rolling(20).mean()
    df['AB_Lower_Band'] = df['ab_down'].rolling(20).mean()

In [126]:
abands(df)

# Drop unwanted columns

In [130]:
df = df.drop(['OpenInt', 'Close_diff','ab_up', 'ab_down'], axis=1)


# Stochastic Oscillator (%K and %D)
тут я вместо $L_{min}$ использовал Avg(L)<br>
__смотри 1 часть__

In [137]:
def stochastic(df, k=14, d=3):
    df['stoch_k'] = ((df['Close'] - df['Low'].rolling(k).mean()) / (df['High'].rolling(k).max() - df['Low'].rolling(k).min())) * 100
    df['stoch_d'] = df['stoch_k'].rolling(d).mean()    

In [139]:
stochastic(df)

# Chaikin Money Flow [CMF](https://www.tradingview.com/wiki/Chaikin_Money_Flow_(CMF)/ru)

### 1. Найдите множитель денежных потоков<br>  $ \frac{(Close  -  Low) - (High - Close)}{(High - Low)} =  Money Flow Multiplier$ 

### 2. Рассчитать объем денежных потоков<br> Money Flow Multiplier x Volume for the Period (Объем за период) = Money Flow Volume 

### 3. Вычислить CMF<br>   Сумма Объема денежных потоков за 21 период / 21 Период суммы объема = 21 период CMF


In [140]:
def CMFlow(df, tf):
    CHMF = []
    MF_Multiplier = []
    MF_Values = []
    x = tf
    #_________________________ [1] Найдите множитель денежных потоков
    while x < len(df['Date']):
        PeriodVolume = 0
        volRange = df['Volume'][x-tf:x]
        for eachVol in volRange:
            PeriodVolume += eachVol
        
        MFM = ((df['Close'][x] - df['Low'][x]) - (df['High'][x] - df['Close'][x])) / (df['High'][x] - df['Low'][x])
        MFV = MFM*PeriodVolume
        
        MFMs.append(MFM)
        MFVs.append(MFV)
        x+=1
    
    y = tf
    while y < len(MFVs):
        PeriodVolume = 0
        volRange = df['Volume'][x-tf:x]
        for eachVol in volRange:
            PeriodVolume += eachVol
        consider = MFVs[y-tf:y]
        tfsMFV = 0
        
        for eachMFV in consider:
            tfsMFV += eachMFV
        
        tfsCMF = tfsMFV/PeriodVolume
        CHMF.append(tfsCMF)
        y+=1
    return CHMF

,Date,Open,High,Low,Close,Volume,RSI_14D,BB_Middle_Band,BB_Upper_Band,BB_Lower_Band,Aroon_25D,PVT,AB_Middle_Band,AB_Upper_Band,AB_Lower_Band,stoch_k,stoch_d
0,2005-02-25,104.77,106.00,104.68,105.79,70221808,0.000000,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN
1,2005-02-28,105.55,105.68,104.56,105.08,79695344,0.000000,NaN,NaN,NaN,NaN,-534868,NaN,NaN,NaN,NaN,NaN
2,2005-03-01,105.22,105.87,105.22,105.62,54607412,0.000000,NaN,NaN,NaN,NaN,-254244,NaN,NaN,NaN,NaN,NaN
3,2005-03-02,105.21,106.22,105.10,105.57,73733090,0.000000,NaN,NaN,NaN,NaN,245719,NaN,NaN,NaN,NaN,NaN
4,2005-03-03,105.99,106.20,105.15,105.61,71286823,0.000000,NaN,NaN,NaN,NaN,-7894.63,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3196,2017-11-06,258.30,259.00,258.22,258.85,49529148,67.174797,254.940,259.882,249.999,96.0,275566,256.3090,258.759908,253.600158,54.035714,50.699383
3197,2017-11-07,258.97,259.35,258.11,258.67,57218448,65.097236,255.260,259.862,250.658,96.0,36866.9,256.5115,258.991980,253.789730,43.651535,51.326967
3198,2017-11-08,258.47,259.22,258.15,259.11,50277699,66.370809,255.562,259.932,251.192,92.0,45734.1,256.7160,259.239105,253.944355,46.802403,48.163218
3199,2017-11-09,257.73,258.39,256.36,258.17,94537282,55.430328,255.823,259.824,251.822,84.0,-257440,256.8925,259.539811,253.911311,29.312417,39.922118
